In [1]:
# basert på https://www.kaggle.com/airalex/recommend-movie-with-clustering

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

data = pd.read_csv("../datasets/imdb-5000-movie-dataset/movie_metadata.csv")

data_use = data.ix[:,['genres','plot_keywords','movie_title','actor_1_name',
                      'actor_2_name','actor_3_name','director_name','imdb_score']]

data_use['movie_title'] = [i.replace("\xc2\xa0","") for i in list(data_use['movie_title'])]

In [3]:
data_use.head(5)

,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score
0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,Avatar,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,7.9
1,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,Gore Verbinski,7.1
2,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Sam Mendes,6.8
3,Action|Thriller,deception|imprisonment|lawlessness|police offi...,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Christopher Nolan,8.5
4,Documentary,NaN,Star Wars: Episode VII - The Force Awakens ...,Doug Walker,Rob Walker,NaN,Doug Walker,7.1


In [4]:
print(data_use.shape)
clean_data = data_use.dropna(axis = 0)
print(clean_data.shape)
clean_data = clean_data.drop_duplicates(['movie_title'])
clean_data = clean_data.reset_index(drop=True)
print(clean_data.shape)

(5043, 8)
(4781, 8)
(4659, 8)


In [5]:
people_list = []
for i in range(clean_data.shape[0]):
    name1 = clean_data.ix[i,'actor_1_name'].replace(" ","_")
    name2 = clean_data.ix[i,'actor_2_name'].replace(" ","_")
    name3 = clean_data.ix[i,'actor_3_name'].replace(" ","_")
    name4 = clean_data.ix[i,'director_name'].replace(" ","_")
    people_list.append("|".join([name1,name2,name3,name4]))
clean_data['people'] = people_list



In [6]:
from sklearn.feature_extraction.text import CountVectorizer
def token(text):
    return(text.split("|"))


cv_kw=CountVectorizer(max_features=100,tokenizer=token )
keywords = cv_kw.fit_transform(clean_data["plot_keywords"])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data["genres"])
genres_list = ["genres_"+ i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data["people"])
people_list = ["pp_"+ i for i in cv_pp.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list

In [7]:
print cluster_data.shape
print  keywords_list

(4659L, 224L)
[u'kw_actor', u'kw_african american', u'kw_alien', u'kw_assassin', u'kw_baby', u'kw_bank', u'kw_bar', u'kw_based on novel', u'kw_battle', u'kw_best friend', u'kw_birthday', u'kw_blood', u'kw_book', u'kw_box office flop', u'kw_boy', u'kw_breasts', u'kw_christmas', u'kw_cia', u'kw_coach', u'kw_college', u'kw_critically bashed', u'kw_cult film', u'kw_death', u'kw_desert', u'kw_detective', u'kw_divorce', u'kw_doctor', u'kw_dog', u'kw_dream', u'kw_drugs', u'kw_escape', u'kw_family relationships', u'kw_fbi', u'kw_female nudity', u'kw_female protagonist', u'kw_fight', u'kw_friend', u'kw_friendship', u'kw_future', u'kw_gang', u'kw_gangster', u'kw_gay', u'kw_ghost', u'kw_girl', u'kw_high school', u'kw_hospital', u'kw_hotel', u'kw_island', u'kw_king', u'kw_lawyer', u'kw_love', u'kw_magic', u'kw_male frontal nudity', u'kw_male nudity', u'kw_marriage', u'kw_martial arts', u'kw_mexico', u'kw_military', u'kw_money', u'kw_monster', u'kw_murder', u'kw_neighbor', u'kw_new york', u'kw_new 

In [8]:
print keywords.todense().shape
print genres.todense().shape
print people.todense().shape

(4659L, 100L)
(4659L, 24L)
(4659L, 100L)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def token(text):
    return(text.split("|"))


cv_kw=CountVectorizer(max_features=100,tokenizer=token )
keywords = cv_kw.fit_transform(clean_data["plot_keywords"])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data["genres"])
genres_list = ["genres_"+ i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data["people"])
people_list = ["pp_"+ i for i in cv_pp.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list

In [10]:
from sklearn.cluster import KMeans

mod = KMeans(n_clusters=100)
category = mod.fit_predict(cluster_data)

category_dataframe = pd.DataFrame({"category":category},index = clean_data['movie_title'])

In [11]:
clean_data.ix[list(category_dataframe['category'] == 0),['genres','movie_title','people']]

,genres,movie_title,people
225,Action|Family|Sport,Speed Racer,Scott_Porter|Kick_Gurry|Nicholas_Elia|Lana_Wac...
257,Action|Drama|Sci-Fi|Sport,Real Steel,Hugh_Jackman|Torey_Michael_Adkins|Olga_Fonda|S...
704,Action|Drama|Sport,Days of Thunder,Tom_Cruise|Robert_Duvall|Randy_Quaid|Tony_Scott
1145,Action|Drama|Family|Sport,The Karate Kid,Martin_Kove|William_Zabka|William_Bassett|John...
1286,Action|Drama|Fantasy,The Crow,Michael_Wincott|Bai_Ling|David_Patrick_Kelly|A...
1301,Action|Biography|Drama,The Grandmaster,Tony_Chiu_Wai_Leung|Hye-kyo_Song|Elvis_Tsui|Ka...
1605,Action|Drama|Thriller,Hanna,Jessica_Barden|Vicky_Krieps|John_Macmillan|Joe...
1664,Action|Drama|Fantasy,The Promise,Dong-gun_Jang|Toby_Leonard_Moore|Nicholas_Tse|...
1948,Action|Drama,Kung Fu Killer,Siu-Wong_Fan|Charlie_Yeung|Wai-Keung_Lau|Teddy...
1955,Action|Drama|Western,Jane Got a Gun,Natalie_Portman|Noah_Emmerich|Boyd_Holbrook|Ga...


In [12]:
def recommend(movie_name,recommend_number = 5):
    if movie_name in list(clean_data['movie_title']):
        movie_cluster = category_dataframe.ix[movie_name,'category']
        score = clean_data.ix[list(category_dataframe['category'] == movie_cluster),['imdb_score','movie_title']]
        sort_score = score.sort_values(['imdb_score'],ascending=[0])
        sort_score = sort_score[sort_score['movie_title'] != movie_name]
        recommend_number = min(sort_score.shape[0],recommend_number)
        recommend_movie = list(sort_score.iloc[range(recommend_number),1])
        print(recommend_movie)
    else:
        print("Can't find this movie!")

In [35]:
recommend('Die Hard with a Vengeance',10)

['Pulp Fiction', 'Die Hard', 'The Sixth Sense', 'The Fifth Element', 'Looper', 'Unbreakable', 'Live Free or Die Hard', 'Die Hard 2', 'RED', 'Beavis and Butt-Head Do America']


In [14]:
list(clean_data['movie_title'])

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz the Great and Powerful',
 'The Amazing Spider-Man 2',


## TODO :
kanskje plukke ut basert på hvilke som er nærmest cluster senter? Eller er imdb score like greit? Kanskje litt visualisering?